- 一回も購入していない人は人気順とかにする？
- 業者っぽいやつだけ別のモデルで学習する
- データの作成過程毎にモデルを分ける
- 入札、お気に入りの分布を見てみる

In [2]:
from os.path import dirname
import os
import datetime
from dateutil.relativedelta import relativedelta
import sys

import pandas as pd
import swifter
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
pd.set_option('display.max_columns', 200)

In [4]:
%reload_ext autoreload
import brandear_est as be

In [5]:
SUB_DIR = os.path.join(os.getcwd(), "../../data/submit/")
IMD_DIR = os.path.join(os.getcwd(), "../../data/intermediate/")
INPUT_DIR = os.path.join(os.getcwd(), "../../data/input")

IMD_INPUT_DIR = os.path.join(IMD_DIR, "pickled_inputs")
IMD_EST_WEEKLY_DIR = os.path.join(IMD_DIR, "rank_weekly")
IMD_ARCHIVE_DIR = os.path.join(IMD_DIR, "arc_rank")


dset_to_period = {
    "train": {"oldest": datetime.datetime(2019, 9, 3, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 10, 0, 0, 0)},
    "valid_for_train": {"oldest": datetime.datetime(2019, 9, 10, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 17, 0, 0, 0),},    
    "valid_for_sub": {"oldest": datetime.datetime(2019, 9, 17, 0, 0, 0),
              "newest": datetime.datetime(2019, 9, 24, 0, 0, 0),},
    "submission": {"oldest": datetime.datetime(2019, 9, 24, 0, 0, 0),
              "newest": datetime.datetime(2019, 10, 1, 0, 0, 0),},    
}

In [6]:
# 元データ読み込み
auction = pd.read_pickle(os.path.join(IMD_INPUT_DIR , "auction.pkl"))
watch = pd.read_pickle(os.path.join(IMD_INPUT_DIR , "watch.pkl"))
bid = pd.read_pickle(os.path.join(IMD_INPUT_DIR , "bid.pkl"))
bid_success = pd.read_pickle(os.path.join(IMD_INPUT_DIR , "bid_success.pkl"))
sample_sub = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))
sub_users = sample_sub[["KaiinID"]].drop_duplicates()

# ブランド作成日
brand = be.read_csv(os.path.join(INPUT_DIR , "brand.csv"))
# itemcolorid付与(色の大分類)
color = be.read_csv(os.path.join(INPUT_DIR , "color.csv"))
# カテゴリーid,itemdailID付与のためのItemShouID付与
genre = be.read_csv(os.path.join(INPUT_DIR , "genre.csv"))
# itemshowID,itemdailID付与
itemshou = be.read_csv(os.path.join(INPUT_DIR , "itemshou.csv"))
# itemlineID付与
line = be.read_csv(os.path.join(INPUT_DIR , "line.csv"))
# 会員登録日、生年月日付与
kaiin = be.read_csv(os.path.join(INPUT_DIR , "kaiin.csv"))
# 入荷お知らせ
nyuuka_oshirase = be.read_csv(os.path.join(INPUT_DIR , "nyuuka_oshirase.csv"))
# 検索履歴
search_log = be.read_csv(os.path.join(INPUT_DIR , "search_log.tsv"), sep="\t")

Mem. usage decreased to  0.32 Mb (15.0% reduction)
Mem. usage decreased to  0.01 Mb (32.2% reduction)
Mem. usage decreased to  0.00 Mb (45.4% reduction)
Mem. usage decreased to  0.00 Mb (33.4% reduction)
Mem. usage decreased to  0.01 Mb (41.3% reduction)
Mem. usage decreased to  5.83 Mb (31.2% reduction)
Mem. usage decreased to 12.62 Mb (43.7% reduction)


/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Mem. usage decreased to 872.24 Mb (40.6% reduction)


In [7]:
# オークション情報拡充
def rename(df, prefix):
    target_columns = ["ModifyDate", "CreateDate"]
    for target_column in target_columns:
        if target_column in df.columns:
            df.rename(columns={target_column: prefix + target_column}, inplace=True)
    return df

itemshou = rename(itemshou, prefix="ItemShow")
genre = rename(genre, prefix="Genre")
brand = rename(brand, prefix="Brand")
color = rename(color, prefix="Color")
line = rename(line, prefix="Line")
kaiin = rename(kaiin, prefix="Kaiin")

genre_mst = (
    genre[["GenreID", "ItemShouID", "CategoryID"]]
    .merge(itemshou[["ItemShouID", "ItemDaiID"]], on="ItemShouID", how="inner")
)

auction_mst = (
    auction
    .merge(genre_mst, on="GenreID", how="left")
    .merge(brand[["BrandID", "BrandCreateDate"]], on="BrandID", how="left")    
    .merge(color[["ColorID", "ItemColorID"]], on="ColorID", how="left")    
    .merge(line[["LineID", "ItemLineID", "LineCreateDate"]], on="LineID", how="left")        
    .fillna(0)
)
auction_mst = be.to_datetime(auction_mst[sorted(list(auction_mst.columns))])

In [8]:
# kaiin["KaiinCreateDate"] = pd.to_datetime(kaiin["KaiinCreateDate"], format='%Y-%m-%d %H:%M:%S')
# search_log["TourokuTime"] = pd.to_datetime(search_log["TourokuTime"], format='%Y-%m-%d %H:%M:%S')
# nyuuka_oshirase = be.to_datetime(nyuuka_oshirase)
# search_log = be.to_datetime(search_log)

In [9]:
def extract_target_actions(watch, bid, period):
    watch_actioned = (
        watch.loc[(watch["TourokuDate"] >= period["oldest"]) & (watch["TourokuDate"] < period["newest"]),
                  ["KaiinID", "AuctionID"]]
    )
    bid_actioned = (
        bid.loc[(bid["ShudouNyuusatsuDate"] >= period["oldest"]) & (bid["ShudouNyuusatsuDate"] < period["newest"]),
                ["KaiinID", "AuctionID"]]
    )
    # 学習用データの際は正解データを作成する
    watch_actioned["watch_actioned"] = 1
    bid_actioned["bid_actioned"] = 1

    target_actions = (
        watch_actioned
        .merge(bid_actioned, on=["KaiinID", "AuctionID"], how="outer")
        .drop_duplicates()
        .fillna(0)
    )

    return target_actions


def arrange_dtime_condition(watch, bid, auction, period):
    retval = (
        watch[watch["TourokuDate"] <= period["oldest"]],
        bid[bid["ShudouNyuusatsuDate"] <= period["oldest"]],
        auction[auction["CreateDate"] < period["newest"]]        
    )
    return retval

def arrange_inputs(watch, bid, auction, period):
    watch_t_arranged, bid_t_arranged, auction_arranged = (
        arrange_dtime_condition(watch=watch, bid=bid, auction=auction, period=period)
    )
    
    retval =  (
        watch_t_arranged.merge(auction, on="AuctionID", how="left"),
        bid_t_arranged.merge(auction, on="AuctionID", how="left"),
        auction_arranged
    )
    return retval

def sort_columns(df):
    return df[sorted(list(df.columns))]

def extract_similar_aucs(target_users, auction, actions,):
    similar_aucs = (
        actions.merge(target_users, on="KaiinID")
            .merge(auction, on="ShouhinID")
    )
    return similar_aucs

In [10]:
def add_user_feature(df, feature_df, col_prefix):
    user_feature = calc_user_feature(feature_df)
    user_feature.columns = [col_prefix + "_KaiinID_" + col if col != "KaiinID" else "KaiinID" for col in user_feature.columns]
    
    return df.merge(user_feature, on="KaiinID", how="left").fillna(0)

def calc_user_feature(feature_df):
    user_feature = (
        feature_df.groupby("KaiinID")
        .agg({
            "AuctionID": {"AuctionID_cnt": "count"},
            "ConditionID": {"ConditionID_mean": "mean"},
            "SaishuppinKaisuu": {
                "SaishuppinKaisuu_mean": "mean",
                "SaishuppinKaisuu_std": "std",
                "SaishuppinKaisuu_sum": "sum"},
            "SankouKakaku": {
                "SankouKakaku_mean": "mean",
                "SankouKakaku_std": "std",
                "SankouKakaku_sum": "sum"}
        })
    ).fillna(0)
    user_feature.columns = user_feature.columns.droplevel(0)
    user_feature = user_feature.reset_index()
    return user_feature

def cross_counts(df, col_set, col_name=None):
    if col_name is not None:
        cnt_col_name = col_name
    elif isinstance(col_set, str):
        cnt_col_name = col_set + "_cnt"
    elif isinstance(col_set, list):
        cnt_col_name = "_".join(col_set) + "_cnt"
    cnts = (
        df.groupby(col_set, as_index=False).size().reset_index()
        .rename(columns={0: cnt_col_name})
    )
    return cnts


In [10]:
# 特徴量作成の設定
dataset_types = ["valid_for_train", "valid_for_sub", "submission"]
# dataset_types = ["valid_for_sub"]
rank_th = 1600
# 商品紐付けとest_rank_weeklyの出力のどちら、または両方使うかのフラグ
inputs_type = ["Shouhin"]
rank_weekly_th = 1600

In [16]:
# %%time

for dset_type in dataset_types:
        
    print(inputs_type)
        
    # データの時系列整理
    watch_arranged, bid_arranged, auction_arranged = (
        arrange_inputs(watch=watch, bid=bid, auction=auction_mst, period=dset_to_period[dset_type])
    )

    target_actions = extract_target_actions(watch=watch, bid=bid, period=dset_to_period[dset_type])    
    target_users = None
    if dset_type == "submission":
        target_users = sub_users
    else:
        target_users = target_actions[["KaiinID"]].drop_duplicates()
    
    # inputとする候補の分岐
    dataset_base = None    
    # est_rank_weeklyの出力のみ使う
    if "rank_weekly" in inputs_type and "Shouhin" not in inputs_type:
        dataset_base = pd.read_pickle(IMD_EST_WEEKLY_DIR + f"/watch_{dset_type}_{rank_th}.pkl")
        dataset_base = dataset_base.query(f"rank <= {rank_weekly_th}")
    # 商品紐付けのみ使う
    elif "rank_weekly" not in inputs_type and "Shouhin" in inputs_type:
        dataset_base = extract_similar_aucs(
            target_users=target_users,
             auction=auction_arranged,
             actions=pd.concat([watch_arranged[["KaiinID", "ShouhinID"]], bid_arranged[["KaiinID", "ShouhinID"]]]).drop_duplicates()
        )
    # 両方使う
    elif "rank_weekly" in inputs_type and "Shouhin" in inputs_type:
        ranked_weekly = pd.read_pickle(IMD_EST_WEEKLY_DIR + f"/watch_{dset_type}_{rank_th}.pkl")
        ranked_weekly = ranked_weekly.query(f"rank <= {rank_weekly_th}")
        similar_aucs = extract_similar_aucs(
            target_users=target_users,
             auction=auction_arranged,
             actions=pd.concat([watch_arranged[["KaiinID", "ShouhinID"]], bid_arranged[["KaiinID", "ShouhinID"]]]).drop_duplicates()
        )    
        dataset_base = (
            pd.concat([ranked_weekly[["AuctionID", "KaiinID"]], similar_aucs[["AuctionID", "KaiinID"]]]).drop_duplicates()
        )
    else:
        raise ValueError()
    
#     if "targets" in inputs_type:
#         dataset_base = pd.concat([dataset_base, target_actions[["KaiinID", "AuctionID"]]], sort=False).drop_duplicates()

    # LDAでブランドをまとめる変数作成
    watch_brand_cross = pd.crosstab(watch_arranged['BrandID'], watch_arranged['KaiinID'])
    watch_brand_pop_cross = watch_brand_cross[watch_brand_cross.sum(axis=1) > 3]
    model = LatentDirichletAllocation(n_components=10)
    lda_pred = pd.DataFrame(model.fit_transform(np.array(watch_brand_pop_cross)))
    brand_lda_category = (pd.DataFrame({
        "BrandID": watch_brand_pop_cross.reset_index()["BrandID"],
        "LDA_Category": lda_pred.idxmax(axis=1).reset_index()[0]
    }))
    auction_arranged = auction_arranged.merge(brand_lda_category, on="BrandID").fillna(11)
    watch_arranged = watch_arranged.merge(brand_lda_category, on="BrandID").fillna(11)
    bid_arranged = bid_arranged.merge(brand_lda_category, on="BrandID").fillna(11)

    # 正解付与
    dataset_base = dataset_base.merge(target_actions, on=["KaiinID", "AuctionID"], how="left").fillna(0)
    
    # オークション情報付与
    dataset_base_a = dataset_base.merge(
        auction_arranged[[col for col in auction_arranged.columns if not col in dataset_base.columns] + ["AuctionID"]],
        on="AuctionID", how="left"
    )

    dataset_base_a = sort_columns(dataset_base_a)
    
    # クロス集計
    w_cate_col = ["AuctionID", "BrandID", "ItemShouID", "LineID", "ShouhinID", "ShouhinShubetsuID", "LDA_Category"]
    b_cate_col = ["AuctionID", "BrandID", "ItemShouID", "ShouhinID", "LDA_Category"]
    def add_cate_with_user(cate_col):
        cate_with_user = [["KaiinID", col] for col in w_cate_col]
        return cate_col + cate_with_user
    
    w_cnt_colsets = add_cate_with_user(w_cate_col)
    b_cnt_colsets = add_cate_with_user(b_cate_col)
    
    dataset_base_cwb = dataset_base_a

    for cross_conf in [[watch_arranged, "watch", w_cnt_colsets], [bid_arranged, "bid", b_cnt_colsets]]:    
        dataset_base_cwb = be.add_cross_counts(
            df=dataset_base_cwb, feature_df=cross_conf[0], prefix=cross_conf[1], col_sets=cross_conf[2]
        )
    
    # ユーザーの特徴量付与
    # 何回watch/bid/successしたか
    # 再出品回数、価格の平均、分散、今回との割合
    dataset_base_u = dataset_base_cwb

    dataset_base_u = add_user_feature(df=dataset_base_u, feature_df=watch_arranged, col_prefix="watch")

    for col in ["SaishuppinKaisuu", "SankouKakaku"]:
        dataset_base_u[f"watch_KaiinID_rate_mean_to_{col}"] = (
            dataset_base_u[f"watch_KaiinID_{col}_mean"] / dataset_base_u[col]
        )    
    
    # 時間系の特緒量付与
    oldest_dtime = dset_to_period[dset_type]["oldest"]
    
    def calc_timedelta(df, dtime_col, delta_col):
        df[delta_col] = df[dtime_col].swifter.apply(lambda d: (oldest_dtime - d).days)
    
    calc_timedelta(dataset_base_u, "CreateDate", "Auction_elapsed_days")
    calc_timedelta(dataset_base_u, "BrandCreateDate", "Brand_elapsed_days")
    calc_timedelta(dataset_base_u, "LineCreateDate", "Line_elapsed_days")
    calc_timedelta(watch_arranged, "TourokuDate", "watch_elapsed_day")    
        
    def agg_time_feature(df, agg_key, na_value):
        time_agg = df.groupby(agg_key).agg({
            "watch_elapsed_day": {
                f"{agg_key}_watch_elapsed_day_max": "max",
                f"{agg_key}_watch_elapsed_day_min": "min",
                f"{agg_key}_watch_elapsed_day_std": "std",
            }    
        }).fillna(na_value)
        time_agg.columns = time_agg.columns.droplevel(0)
        time_agg = time_agg.reset_index()
        return time_agg
        
    w_k_d = agg_time_feature(df=watch_arranged, agg_key="KaiinID", na_value=999)
    w_a_d = agg_time_feature(df=watch_arranged, agg_key="AuctionID", na_value=0)
    
    dataset_base_u = (
        dataset_base_u.merge(w_k_d, on="KaiinID", how="left").fillna(999)
        .merge(w_a_d, on="AuctionID", how="left").fillna(0)
    )
    
    # お気に入り合計に対する該当オークションのお気に入り数

    dataset_base_b = dataset_base_u
    dataset_base_b["watch_BrandID_KaiinID_rate"] = (
        dataset_base_b["watch_KaiinID_BrandID_cnt"] / dataset_base_b["watch_KaiinID_AuctionID_cnt_y"]
    )
    brand_ave = (
        auction_arranged[["BrandID", "SankouKakaku"]].groupby("BrandID", as_index=False).mean()
        .rename(columns={"SankouKakaku": "BrandID_SankouKakaku"})
    )
    dataset_base_b = dataset_base_b.merge(brand_ave, on="BrandID", how="left")
    dataset_base_b["SankouKakaku_rate_to_BrandID"] = (
        dataset_base_b["SankouKakaku"] / dataset_base_b["BrandID_SankouKakaku"]
    )
    
    now = datetime.datetime.now().strftime("%Y%m%d%H%M")
    be.df2pkl(dataset_base_b, IMD_ARCHIVE_DIR, f"{dset_type}_feature_{rank_th}_{now}.pkl")    

['Shouhin']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'LineID', 'ShouhinID', 'ShouhinShubetsuID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
LineID
ShouhinID
ShouhinShubetsuID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID', 'ItemShouID']
['KaiinID', 'LineID']
['KaiinID', 'ShouhinID']
['KaiinID', 'ShouhinShubetsuID']
['KaiinID', 'LDA_Category']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'ShouhinID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
ShouhinID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID'

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


['Shouhin']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'LineID', 'ShouhinID', 'ShouhinShubetsuID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
LineID
ShouhinID
ShouhinShubetsuID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID', 'ItemShouID']
['KaiinID', 'LineID']
['KaiinID', 'ShouhinID']
['KaiinID', 'ShouhinShubetsuID']
['KaiinID', 'LDA_Category']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'ShouhinID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
ShouhinID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID'

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


['Shouhin']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'LineID', 'ShouhinID', 'ShouhinShubetsuID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
LineID
ShouhinID
ShouhinShubetsuID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID', 'ItemShouID']
['KaiinID', 'LineID']
['KaiinID', 'ShouhinID']
['KaiinID', 'ShouhinShubetsuID']
['KaiinID', 'LDA_Category']
##################
start cross count
['AuctionID', 'BrandID', 'ItemShouID', 'ShouhinID', 'LDA_Category', ['KaiinID', 'AuctionID'], ['KaiinID', 'BrandID'], ['KaiinID', 'ItemShouID'], ['KaiinID', 'LineID'], ['KaiinID', 'ShouhinID'], ['KaiinID', 'ShouhinShubetsuID'], ['KaiinID', 'LDA_Category']]
AuctionID
BrandID
ItemShouID
ShouhinID
LDA_Category
['KaiinID', 'AuctionID']
['KaiinID', 'BrandID']
['KaiinID'

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


In [11]:
import numpy as np
import pandas as pd
import lightgbm as lgb


class LgbLambdaLank:
    def __init__(self, params=None):
        self.model = None
        if params is None:
            params = {
                'objective': 'lambdarank',
                'metric': 'ndcg',
                "ndcg_at": 20,
                "nround": 500,
                "learning_rate": 0.01,
                "max_depth": 6,
                "num_leaves": 127
            }
        self.params = params

    @staticmethod
    def _adjust_data(dataset):
        dataset.data.sort_values(["KaiinID", "AuctionID"], inplace=True)
        if {"watch_actioned", "bid_actioned"} - set(dataset.data.columns) == set([]):
            label = np.array(dataset.data[["watch_actioned", "bid_actioned"]].astype(int)).max(axis=1)
        else:
            label = None

        weight = (
            np.stack([
                np.array(dataset.data["watch_actioned"].astype(int)),
                (np.array(dataset.data["bid_actioned"]).astype(int) * 2),
                np.ones((dataset.data.shape[0],))
            ], 1).max(axis=1)
        )

        group = (
            dataset.data[["KaiinID", "AuctionID"]]
            .groupby("KaiinID", as_index=False)
            .count()
            .sort_values("KaiinID")["AuctionID"]
        )

        lgb_dataset = lgb.Dataset(
            data=np.array(dataset.drop()),
            label=label,
            weight=weight,
            group=group
        )

        return lgb_dataset

    def train(self, train_dataset, valid_dataset=None, desc=False):
        self.model = lgb.train(
            params=self.params,
            train_set=self._adjust_data(train_dataset),
            valid_sets=self._adjust_data(valid_dataset) if valid_dataset is not None else None
        )
        if desc:
            print("\n\n##################")
            print(self.get_model_info(train_dataset))
            print("##################")

    def predict(self, dataset):
        dataset.data.sort_values(["KaiinID", "AuctionID"], inplace=True)
        return self.model.predict(
            data=np.array(dataset.drop()),
            group=np.array(dataset.data[["KaiinID", "AuctionID"]].groupby("KaiinID", as_index=False).count()
                           .sort_values("KaiinID")["AuctionID"])
        )

    def get_model_info(self, dataset):
        importance = pd.DataFrame(
            self.model.feature_importance(),
            index=dataset.drop().columns,
            columns=['importance']
        )
        return importance


In [12]:

def comple_submit_auc(df):
    target_users = df.groupby("KaiinID", as_index=False).count().query("score < 20")["KaiinID"].tolist()
    if target_users == []:
        return df
    else:
        candidate_aucs = (
            df[["AuctionID", "score"]]
            .groupby("AuctionID", as_index=False).mean().sort_values("score", ascending=False).iloc[:40, :]
        )
        candidate_aucs["score"] = -999
        buf = []
        for user in target_users:
            candidate_aucs_tmp = candidate_aucs.copy()
            candidate_aucs_tmp["KaiinID"] = user
            buf.append(candidate_aucs_tmp)
        df_comple = pd.concat(buf)
        df_colmled = pd.concat([df, df_comple], sort=False)
        return df_colmled


def adjust_sub_form(users, pred, drop=False):
    sub_data = users.merge(pred, on="KaiinID", how="left")[["KaiinID", "AuctionID", "score"]]
    sub_data = comple_submit_auc(sub_data)
    sub_data.sort_values(['KaiinID', 'score'], ascending=[True, False], inplace=True)
    sub_data['rank'] = sub_data.groupby('KaiinID')['score'].cumcount()
    sub_valid = sub_data.query("rank < =19")
    sub_valid = sub_valid.sort_values(['KaiinID', 'score'], ascending=[True, False])
    if drop:
        sub_valid.drop(["score", "rank"], axis=1, inplace=True)

    return sub_valid


def get_cheat_pred(data, target_actions):
    actiones = data[["KaiinID", "AuctionID"]].copy()
    scored_targets = (
        stack_target_actions(target_actions).groupby(["KaiinID", "AuctionID"], as_index=False).max())
    cheat_pred = (
        actiones.merge(scored_targets, on=["KaiinID", "AuctionID"], how="left")
        .fillna(0).sort_values(["KaiinID", "score"], ascending=["True", "False"]))
    return cheat_pred

def stack_target_actions(target_actions):
    watch_target = target_actions.query("(watch_actioned == 1)")[["KaiinID", "AuctionID"]]
    bid_target = target_actions.query("(bid_actioned == 1)")[["KaiinID", "AuctionID"]]
    watch_target["score"] = 1
    bid_target["score"] = 2
    stacked_target_actions = pd.concat([watch_target, bid_target], sort=False)
    return stacked_target_actions


def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    return np.sum((2 ** r - 1) / np.log2(np.arange(2, r.size + 2)))


def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

def calc_ndcg(y_true, y_pred, k=20):
    y_pred_cp = y_pred.copy()
    
    actione_true = stack_target_actions(y_true)
    actione_true["rank"] = 100

    y_pred_cp['rank'] = y_pred_cp.groupby('KaiinID')['AuctionID'].cumcount()

    scored_pred = (
        y_pred_cp.merge(actione_true[["KaiinID", "AuctionID", "score"]], on=["KaiinID", "AuctionID"],
                     how="left").fillna(0))

    unchoiced_actiones = (
        be.left_anti_join(actione_true, y_pred_cp, ["KaiinID", "AuctionID"], ["KaiinID", "AuctionID"]))

    scored_actiones = (
        pd.concat([scored_pred, unchoiced_actiones], sort=False)
        .sort_values(["KaiinID", "rank"], ascending=["True", "True"]))
    
    dcgs = scored_actiones.groupby("KaiinID")["score"].apply(lambda s: ndcg_at_k(s.tolist(), k=20))
    ndcg = dcgs.mean()

    return ndcg


In [22]:
# vt_dataset_base = pd.read_pickle(IMD_ARCHIVE_DIR + "/valid_for_sub_feature_1600_shouhin_plus_1600_cands.pkl")
vs_dataset_base = pd.read_pickle(IMD_ARCHIVE_DIR + "/valid_for_sub_feature_1600_shouhin_plus_1600_cands.pkl")
# sub_dataset_base = pd.read_pickle(IMD_ARCHIVE_DIR + "/submission_feature_1600_shouhin.pkl")

auc_foreign_ids = ["BrandID", "CategoryID", "ColorID", "ConditionID", "DanjobetsuID", "GenreGroupID",
            "GenreID", "ItemColorID", "ItemDaiID", "ItemLineID", "ItemShouID", "LineID", "ShouhinID", "ShouhinShubetsuID"]
drop_cols = (
    ["KaiinID", "AuctionID", "BrandCreateDate", "LineCreateDate", "CreateDate", "rank", "watch_actioned", "bid_actioned"]
    + auc_foreign_ids
)
# watch_vt_dataset = be.DataSet(data=vt_dataset_base, drop_cols=drop_cols, target_col="watch_actioned")
# watch_vs_dataset = be.DataSet(data=vs_dataset_base, drop_cols=drop_cols, target_col="watch_actioned")
# watch_vs_dataset = be.DataSet(
#     data=pd.concat([vt_dataset_base, vs_dataset_base]),
#     drop_cols=drop_cols,
#     target_col="watch_actioned"
# )
# watch_sub_dataset = be.DataSet(data=sub_dataset_base, drop_cols=drop_cols, target_col="watch_actioned")
# bid_vs_dataset = be.DataSet(data=vs_dataset_base, drop_cols=drop_cols, target_col="bid_actioned")

In [24]:
dset_type = "valid_for_sub"
watch_arranged, bid_arranged, auction_arranged = (
    arrange_inputs(watch=watch, bid=bid, auction=auction_mst, period=dset_to_period[dset_type])
)
target_actions = extract_target_actions(watch=watch, bid=bid, period=dset_to_period[dset_type])    

similar_aucs = extract_similar_aucs(
    target_users=target_actions[["KaiinID"]].drop_duplicates(),
     auction=auction_arranged,
     actions=pd.concat([watch_arranged[["KaiinID", "ShouhinID"]], bid_arranged[["KaiinID", "ShouhinID"]]]).drop_duplicates()
)

similar_aucs = similar_aucs[["KaiinID", "AuctionID"]]
similar_aucs["is_from_similar"] = 1

vs_dataset_base = vs_dataset_base.merge(similar_aucs, on=["KaiinID", "AuctionID"], how="left").fillna(0)

In [38]:
vs_dataset_base = vs_dataset_base.merge(similar_aucs, on=["KaiinID", "AuctionID"], how="left").fillna(0)

In [39]:
vs_dataset_base.head()

,AuctionID,BrandCreateDate,BrandID,CategoryID,ColorID,ConditionID,CreateDate,DanjobetsuID,GenreGroupID,GenreID,ItemColorID,ItemDaiID,ItemLineID,ItemShouID,KaiinID,LineCreateDate,LineID,SaishuppinKaisuu,SankouKakaku,ShouhinID,ShouhinShubetsuID,bid_actioned,watch_actioned,watch_AuctionID_cnt,watch_BrandID_cnt,watch_ItemShouID_cnt,watch_LineID_cnt,watch_ShouhinID_cnt,watch_ShouhinShubetsuID_cnt,watch_KaiinID_AuctionID_cnt_x,watch_KaiinID_BrandID_cnt,watch_KaiinID_ItemShouID_cnt,watch_KaiinID_LineID_cnt,watch_KaiinID_ShouhinID_cnt,watch_KaiinID_ShouhinShubetsuID_cnt,bid_AuctionID_cnt,bid_BrandID_cnt,bid_ItemShouID_cnt,bid_ShouhinID_cnt,bid_KaiinID_AuctionID_cnt,bid_KaiinID_BrandID_cnt,bid_KaiinID_ItemShouID_cnt,bid_KaiinID_LineID_cnt,bid_KaiinID_ShouhinID_cnt,bid_KaiinID_ShouhinShubetsuID_cnt,watch_KaiinID_AuctionID_cnt_y,watch_KaiinID_ConditionID_mean,watch_KaiinID_SaishuppinKaisuu_mean,watch_KaiinID_SaishuppinKaisuu_std,watch_KaiinID_SaishuppinKaisuu_sum,watch_KaiinID_SankouKakaku_mean,watch_KaiinID_SankouKakaku_std,watch_KaiinID_SankouKakaku_sum,watch_KaiinID_rate_mean_to_SaishuppinKaisuu,watch_KaiinID_rate_mean_to_SankouKakaku,Auction_elapsed_days,Brand_elapsed_days,Line_elapsed_days,KaiinID_watch_elapsed_day_max,KaiinID_watch_elapsed_day_min,KaiinID_watch_elapsed_day_std,AuctionID_watch_elapsed_day_max,AuctionID_watch_elapsed_day_min,AuctionID_watch_elapsed_day_std,watch_BrandID_KaiinID_rate,BrandID_SankouKakaku,SankouKakaku_rate_to_BrandID,is_from_similar
0,2648756,2010-09-10 12:06:39,83,1.0,6,6,2019-07-13 09:41:29,0,312,341,1.0,1.0,0.0,3.0,85080,1970-01-01,0,320,66200,756675,1,0.0,0.0,3.0,83549.0,274843,3874863.0,6.0,3979439,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10222.0,32354,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,2500.0,0.0,2500.0,0.0,0.037764,65,3293,18156,36.0,36.0,999.0,65.0,15.0,28.867513,0.0,17855.269692,3.707589,0.0
1,2717891,2010-09-10 12:06:39,83,1.0,15,6,2019-07-02 09:42:03,0,312,331,2.0,1.0,0.0,5.0,85080,1970-01-01,0,309,65100,541424,1,0.0,0.0,7.0,83549.0,62511,3874863.0,14.0,3979439,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10222.0,6319,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,2500.0,0.0,2500.0,0.0,0.038402,76,3293,18156,36.0,36.0,999.0,76.0,26.0,19.042559,0.0,17855.269692,3.645982,0.0
2,2718164,2010-09-10 12:06:39,56,4.0,0,7,2019-08-05 09:30:10,0,295,295,0.0,3.0,0.0,30.0,85080,1970-01-01,0,343,65100,446613,1,0.0,0.0,7.0,53218.0,104174,3874863.0,7.0,3979439,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4576.0,9873,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,2500.0,0.0,2500.0,0.0,0.038402,42,3293,18156,36.0,36.0,999.0,42.0,9.0,12.932057,0.0,14228.507160,4.575322,0.0
3,2673948,2010-09-10 12:06:39,12,11.0,68,7,2019-07-13 09:41:05,2,38,38,4.0,9.0,0.0,62.0,85080,1970-01-01,0,320,63000,132065,3,0.0,0.0,9.0,148682.0,99925,3874863.0,16.0,99925,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16704.0,13047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,8.0,0.0,0.0,0.0,2500.0,0.0,2500.0,0.0,0.039683,65,3293,18156,36.0,36.0,999.0,65.0,13.0,16.686655,0.0,40845.616706,1.542393,0.0
4,2641421,2010-09-10 12:06:39,19,4.0,36,7,2019-07-31 09:35:19,0,295,295,3.0,3.0,0.0,30.0,85080,1970-01-01,0,338,72500,757460,1,0.0,0.0,3.0,33103.0,104174,3874863.0,3.0,3979439,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3206.0,9873,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,2500.0,0.0,2500.0,0.0,0.034483,47,3293,18156,36.0,36.0,999.0,43.0,5.0,19.000000,0.0,40770.154967,1.778262,0.0


In [40]:
auc_foreign_ids = ["BrandID", "CategoryID", "ColorID", "ConditionID", "DanjobetsuID", "GenreGroupID",
            "GenreID", "ItemColorID", "ItemDaiID", "ItemLineID", "ItemShouID", "LineID", "ShouhinID", "ShouhinShubetsuID"]
drop_cols = (
    ["KaiinID", "AuctionID", "BrandCreateDate", "LineCreateDate", "CreateDate", "rank", "watch_actioned", "bid_actioned"]
    + auc_foreign_ids
)

watch_vs_dataset = be.DataSet(data=vs_dataset_base, drop_cols=drop_cols, target_col="is_from_similar")

In [ ]:
# # 学習の様子可視化
# train test split
train_dataset, test_dataset =be.DataSet.train_test_split(watch_vs_dataset)
# sampled_train = be.DataSet.under_sampling(train_dataset)

# target encoding
# cat_col = "BrandID"
# sampled_train.add_target_encode(cat_col=cat_col)
# be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    

lgb_bi = be.LgbBinaryClassifier()
lgb_bi.train(train_dataset=train_dataset, valid_dataset=test_dataset, desc=True)    

In [ ]:
# # 学習の様子可視化
# train test split
train_dataset, test_dataset =be.DataSet.train_test_split(dataset)
sampled_train = be.DataSet.under_sampling(train_dataset)

# target encoding
cat_col = "BrandID"
sampled_train.add_target_encode(cat_col=cat_col)
be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    

lgb_bi = be.LgbBinaryClassifier()
lgb_bi.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)    

In [ ]:
# train test split
train_dataset, test_dataset =be.DataSet.train_test_split(watch_vs_dataset)

lgb_rank = LgbLambdaLank()

if sample_flg:
sampled_train = be.DataSet.under_sampling(train_dataset, rate=100)

# target encoding
cat_cols = ["BrandID", "GenreID"]
for cat_col in cat_cols:
    sampled_train.add_target_encode(cat_col=cat_col)
    be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    

lgb_rank.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)


In [14]:
# ランダムな予測の場合
y_true = extract_target_actions(watch, bid, dset_to_period["valid_for_sub"])
random_pred = watch_vs_dataset.data[["KaiinID", "AuctionID"]].copy()
random_pred["score"] = pd.Series(np.random.random(len(random_pred)), index=random_pred.index)
random_sub = adjust_sub_form(y_true[["KaiinID"]].drop_duplicates(), random_pred, drop=True)
ndcg_score = calc_ndcg(y_true, random_sub)
print("ndcg_score : ", ndcg_score)

ndcg_score :  0.014145549877355328


In [15]:
# 手元のデータで最善の予測をした際の結果
y_true = extract_target_actions(watch, bid, dset_to_period["valid_for_sub"])
cheat_pred = get_cheat_pred(watch_vs_dataset.data, y_true)
cheat_sub = adjust_sub_form(y_true[["KaiinID"]].drop_duplicates(), cheat_pred, drop=True)
# cheat_sub = be.adjust_sub_form(choiced_cands[["KaiinID"]].drop_duplicates(), cheat_pred, drop=True)
ndcg_score = calc_ndcg(y_true, cheat_sub)
print("ndcg_score : ", ndcg_score)

ndcg_score :  0.1553342146674732


In [ ]:
%%time
# watch学習

dataset_pairs = [
    [watch_vt_dataset, watch_vs_dataset]
]

sample_flg = True

# 学習の様子可視化
for in_train_dataset, sub_dataset in dataset_pairs:
    
    # train test split
    train_dataset, test_dataset =be.DataSet.train_test_split(in_train_dataset)
    
    lgb_rank = LgbLambdaLank()
    
    if sample_flg:
        sampled_train = be.DataSet.under_sampling(train_dataset, rate=100)

        # target encoding
        cat_cols = ["BrandID", "GenreID"]
        for cat_col in cat_cols:
            sampled_train.add_target_encode(cat_col=cat_col)
            be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    
    
        lgb_rank.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)

    else:
        # target encoding
        cat_cols = ["BrandID", "GenreID"]
        for cat_col in cat_cols:
            train_dataset.add_target_encode(cat_col=cat_col)
            be.target_encode_for_test(train_dataset=train_dataset, test_dataset=test_dataset, cat_col=cat_col)    
        lgb_rank.train(train_dataset=train_dataset, valid_dataset=test_dataset, desc=True)        

    for cat_col in cat_cols:
        be.target_encode_for_test(train_dataset=in_train_dataset, test_dataset=sub_dataset, cat_col=cat_col)        
    sub_dataset.set_pred(lgb_rank.predict(sub_dataset))    

In [ ]:
# 手元のデータセットでndcg計算
y_true = be.extract_target_actions(watch, bid, dset_to_period["valid_for_sub"])
sub = adjust_sub_form(y_true[["KaiinID"]].drop_duplicates(),
                      watch_vs_dataset.data[["KaiinID", "AuctionID", "pred"]].rename(columns={"pred":"score"}), drop=True
)
ndcg_score = be.calc_ndcg(y_true, sub)
print("ndcg_score : ", ndcg_score)

In [ ]:
tmp = lgb_rank.get_model_info(watch_vs_dataset)
tmp.sort_values("importance", ascending=False).head(30)

In [ ]:
%%time
# 予測値取得
dataset_pairs = [
    [watch_vs_dataset, watch_sub_dataset]
]

rank_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    "ndcg_at": 20,
    "nround": 500,
    "learning_rate": 0.01,
    "max_depth": 6,
    "num_leaves": 127,
    "num_iteration": 500
}

sample_flg = False

# 学習の様子可視化
for in_train_dataset, sub_dataset in dataset_pairs:

    # train test split
    train_dataset, test_dataset =be.DataSet.train_test_split(in_train_dataset)
    
    lgb_rank = LgbLambdaLank()    
    
    if sample_flg:
        sampled_train = be.DataSet.under_sampling(train_dataset, rate=100)

        # target encoding
        cat_cols = ["BrandID", "GenreID"]
        for cat_col in cat_cols:
            sampled_train.add_target_encode(cat_col=cat_col)
            be.target_encode_for_test(train_dataset=train_dataset, test_dataset=test_dataset, cat_col=cat_col)            
        lgb_rank.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)
    else:
        # target encoding
        cat_cols = ["BrandID", "GenreID"]
        for cat_col in cat_cols:
            train_dataset.add_target_encode(cat_col=cat_col)
            be.target_encode_for_test(train_dataset=train_dataset, test_dataset=test_dataset, cat_col=cat_col)    
        lgb_rank.train(train_dataset=train_dataset, valid_dataset=test_dataset, desc=True)        

    for cat_col in cat_cols:
        be.target_encode_for_test(train_dataset=in_train_dataset, test_dataset=sub_dataset, cat_col=cat_col)        
    sub_dataset.set_pred(lgb_rank.predict(sub_dataset))    

In [ ]:
tmp = lgb_rank.get_model_info(sub_dataset)
tmp.sort_values("importance", ascending=False).head(30)

In [24]:
def comple_submit_auc(df):
    target_users = df.groupby("KaiinID", as_index=False).count().query("score < 20")["KaiinID"].tolist()
    if target_users == []:
        return df
    else:
        candidate_aucs = (
            df[["AuctionID", "score"]]
            .groupby("AuctionID", as_index=False).mean().sort_values("score", ascending=False).iloc[:40, :]
        )
        candidate_aucs["score"] -=  999
        buf = []
        for user in target_users:
            candidate_aucs_tmp = candidate_aucs.copy()
            candidate_aucs_tmp["KaiinID"] = user
            buf.append(candidate_aucs_tmp)
        df_comple = pd.concat(buf)
        df_colmled = pd.concat([df, df_comple], sort=False).groupby(["KaiinID", "AuctionID"], as_index=False).max()
        return df_colmled

In [25]:
pred = watch_sub_dataset.data[["KaiinID", "AuctionID", "pred"]].rename(columns={"pred": "score"})
sub = adjust_sub_form(sub_users, pred, drop=True)
sub.to_csv(SUB_DIR + datetime.datetime.now().strftime("%Y%m%d%H%M") + "_submit.csv", index=False)
sub.head()

,KaiinID,AuctionID
0,24,19614.0
1,24,135675.0
5,24,449022.0
7,24,705584.0
8,24,791957.0


In [64]:
%%time
# アンサンブル
# watch学習

dataset_pairs = [
    [watch_vt_dataset, watch_vs_dataset]
]

# 学習の様子可視化
for in_train_dataset, sub_dataset in dataset_pairs:
    
    # train test split
    undersampled_datasets = [
        {"train_dataset": splited[0], "valid_dataset": splited[1]}
        for splited in 
        [be.DataSet.train_test_split(be.DataSet.under_sampling(in_train_dataset, rate=100))
        for _ in range(4)]
    ]
        
    # target encoding
#     cat_cols = ["BrandID", "GenreID"]
#     for cat_col in cat_cols:
#         sampled_train.add_target_encode(cat_col=cat_col)
#         be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    
    
    models = be.DataEmsembler(LgbLambdaLank)
    models.train(undersampled_datasets)
    
    sub_dataset.set_pred(models.predict(sub_dataset))    

[1]	valid_0's ndcg@20: 0.892296
[2]	valid_0's ndcg@20: 0.900662
[3]	valid_0's ndcg@20: 0.902642
[4]	valid_0's ndcg@20: 0.902543
[5]	valid_0's ndcg@20: 0.903634
[6]	valid_0's ndcg@20: 0.90407
[7]	valid_0's ndcg@20: 0.903476
[8]	valid_0's ndcg@20: 0.906179
[9]	valid_0's ndcg@20: 0.906513
[10]	valid_0's ndcg@20: 0.906211
[11]	valid_0's ndcg@20: 0.906645
[12]	valid_0's ndcg@20: 0.906717
[13]	valid_0's ndcg@20: 0.906789
[14]	valid_0's ndcg@20: 0.906407
[15]	valid_0's ndcg@20: 0.906738
[16]	valid_0's ndcg@20: 0.907029
[17]	valid_0's ndcg@20: 0.907427
[18]	valid_0's ndcg@20: 0.907248
[19]	valid_0's ndcg@20: 0.907019
[20]	valid_0's ndcg@20: 0.907291
[21]	valid_0's ndcg@20: 0.90717
[22]	valid_0's ndcg@20: 0.907242
[23]	valid_0's ndcg@20: 0.907145
[24]	valid_0's ndcg@20: 0.907292
[25]	valid_0's ndcg@20: 0.907445
[26]	valid_0's ndcg@20: 0.907496
[27]	valid_0's ndcg@20: 0.90757
[28]	valid_0's ndcg@20: 0.90753
[29]	valid_0's ndcg@20: 0.907704
[30]	valid_0's ndcg@20: 0.907758
[31]	valid_0's ndcg@20:

In [56]:
%%time
# bid学習

# 学習の様子可視化
for target, dataset in {"bid": bid_vs_dataset}.items():
    print(target)
    
    # train test split
    train_dataset, test_dataset =be.DataSet.train_test_split(dataset)
    sampled_train = be.DataSet.under_sampling(train_dataset)

    # target encoding
#     cat_cols = ["BrandID", "GenreID"]
#     # userも組み合わせる
#     for cat_col in cat_cols:
#         sampled_train.add_target_encode(cat_col=cat_col)
#         be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    
    
    lgb_bi = be.LgbBinaryClassifier()
    lgb_bi.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)    

bid
[1]	valid_0's auc: 0.967723
[2]	valid_0's auc: 0.969615
[3]	valid_0's auc: 0.969867
[4]	valid_0's auc: 0.97008
[5]	valid_0's auc: 0.970028
[6]	valid_0's auc: 0.970128
[7]	valid_0's auc: 0.970536
[8]	valid_0's auc: 0.970549
[9]	valid_0's auc: 0.970813
[10]	valid_0's auc: 0.970847
[11]	valid_0's auc: 0.970968
[12]	valid_0's auc: 0.971024
[13]	valid_0's auc: 0.971418
[14]	valid_0's auc: 0.971452
[15]	valid_0's auc: 0.97176
[16]	valid_0's auc: 0.971855
[17]	valid_0's auc: 0.971855
[18]	valid_0's auc: 0.971889
[19]	valid_0's auc: 0.971999
[20]	valid_0's auc: 0.972026
[21]	valid_0's auc: 0.971983
[22]	valid_0's auc: 0.972125
[23]	valid_0's auc: 0.972186
[24]	valid_0's auc: 0.97229
[25]	valid_0's auc: 0.972349
[26]	valid_0's auc: 0.972334
[27]	valid_0's auc: 0.972438
[28]	valid_0's auc: 0.971929
[29]	valid_0's auc: 0.971969
[30]	valid_0's auc: 0.971986
[31]	valid_0's auc: 0.971998
[32]	valid_0's auc: 0.97201
[33]	valid_0's auc: 0.972008
[34]	valid_0's auc: 0.97202
[35]	valid_0's auc: 0.97

In [57]:
tmp = lgb_bi.get_model_info(sampled_train)
tmp.sort_values("importance", ascending=False).head(30)

,importance
AuctionID_watch_elapsed_day_min,650
SaishuppinKaisuu,402
BrandID_SankouKakaku,370
watch_ShouhinID_cnt,341
Auction_elapsed_days,309
AuctionID_watch_elapsed_day_max,302
KaiinID_watch_elapsed_day_min,270
watch_BrandID_cnt,245
bid_AuctionID_cnt,203
bid_KaiinID_BrandID_cnt,194


In [58]:
%%time
# watch学習
# params = {
#     "objective": "binary",
#     'metric': 'binary_logloss',
#     "nround": 500,
#     "learning_rate": 0.01,
#     "max_depth": 6,
#     "num_leaves": 127
# }
# 学習の様子可視化
for target, dataset in {"watch": watch_vs_dataset}.items():
    print(target)
    
    # train test split
    train_dataset, test_dataset =be.DataSet.train_test_split(dataset)
    sampled_train = be.DataSet.under_sampling(train_dataset)

    # target encoding
    cat_cols = ["BrandID", "GenreID"]
    for cat_col in cat_cols:
        sampled_train.add_target_encode(cat_col=cat_col)
        be.target_encode_for_test(train_dataset=sampled_train, test_dataset=test_dataset, cat_col=cat_col)    
    
    lgb_bi = be.LgbBinaryClassifier()
    lgb_bi.train(train_dataset=sampled_train, valid_dataset=test_dataset, desc=True)


watch
[1]	valid_0's auc: 0.952835
[2]	valid_0's auc: 0.953211
[3]	valid_0's auc: 0.956015
[4]	valid_0's auc: 0.956183
[5]	valid_0's auc: 0.956196
[6]	valid_0's auc: 0.956116
[7]	valid_0's auc: 0.956201
[8]	valid_0's auc: 0.956405
[9]	valid_0's auc: 0.956599
[10]	valid_0's auc: 0.956641
[11]	valid_0's auc: 0.956557
[12]	valid_0's auc: 0.956557
[13]	valid_0's auc: 0.956544
[14]	valid_0's auc: 0.956488
[15]	valid_0's auc: 0.956492
[16]	valid_0's auc: 0.956395
[17]	valid_0's auc: 0.956541
[18]	valid_0's auc: 0.956532
[19]	valid_0's auc: 0.956653
[20]	valid_0's auc: 0.956694
[21]	valid_0's auc: 0.956825
[22]	valid_0's auc: 0.95692
[23]	valid_0's auc: 0.957064
[24]	valid_0's auc: 0.957152
[25]	valid_0's auc: 0.957189
[26]	valid_0's auc: 0.957338
[27]	valid_0's auc: 0.957423
[28]	valid_0's auc: 0.957503
[29]	valid_0's auc: 0.95762
[30]	valid_0's auc: 0.957656
[31]	valid_0's auc: 0.957674
[32]	valid_0's auc: 0.957747
[33]	valid_0's auc: 0.957871
[34]	valid_0's auc: 0.957904
[35]	valid_0's auc:

In [59]:
tmp = lgb_bi.get_model_info(sampled_train)
tmp.sort_values("importance", ascending=False).head(30)

,importance
AuctionID_watch_elapsed_day_min,420
BrandID_target_mean,323
Auction_elapsed_days,284
watch_KaiinID_AuctionID_cnt_x,260
watch_KaiinID_SaishuppinKaisuu_sum,212
bid_AuctionID_cnt,197
SaishuppinKaisuu,178
watch_KaiinID_ShouhinID_cnt,165
watch_KaiinID_BrandID_cnt,163
KaiinID_watch_elapsed_day_min,162
